In [1]:
from utils.ds_clean import clean_ds
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split


2024-04-02 12:52:24.860659: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 12:52:24.860941: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 12:52:24.863745: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 12:52:24.894975: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-02 12:52:25.704892: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
df1 = pd.read_csv("data/aapl_5m_train.csv")
df_5min = clean_ds(df1)


In [3]:
df_5min

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,X_t-1,X_t-2,X_t-3,X_t-4,X_t-5,Pt_5,RSI,Y_BUY,Y_SELL
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0,NaN,NaN,NaN,NaN,NaN,131.448898,NaN,False,True
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0,132.809997,NaN,NaN,NaN,NaN,131.119995,NaN,False,True
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0,131.889999,132.809997,NaN,NaN,NaN,130.875000,NaN,False,True
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0,132.059997,131.889999,132.809997,NaN,NaN,130.509902,NaN,False,True
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0,132.250000,132.059997,131.889999,132.809997,NaN,130.889999,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39574,19788,1672432800,0,2022-12-30 20:40:00,129.029998,129.130004,128.919998,129.054992,833767.0,129.039993,129.020004,128.910003,128.940002,128.890106,NaN,56.509324,False,False
39575,19789,1672433100,0,2022-12-30 20:45:00,129.059997,129.399993,129.020004,129.380004,2136910.0,129.054992,129.039993,129.020004,128.910003,128.940002,NaN,60.700323,False,False
39576,19790,1672433400,0,2022-12-30 20:50:00,129.375000,129.929992,129.330001,129.764404,2879267.0,129.380004,129.054992,129.039993,129.020004,128.910003,NaN,64.854367,False,False
39577,19791,1672433700,0,2022-12-30 20:55:00,129.764999,129.949996,129.619995,129.949996,3896812.0,129.764404,129.380004,129.054992,129.039993,129.020004,NaN,66.620919,False,False


In [4]:
close_data_buy = df_5min[['Close', 'X_t-1', 'X_t-2', 'X_t-3', 'X_t-4' ,'X_t-5','RSI', 'Y_BUY']]
close_data_buy = close_data_buy.dropna()

In [6]:
close_data_buy.to_csv('./data/close_data_buy_5.csv', index=False)

In [7]:
close_data_sell = df_5min[['Close', 'X_t-1', 'X_t-2', 'X_t-3', 'X_t-4' ,'RSI', 'Y_SELL']]
close_data_sell = close_data_sell.dropna()

In [ ]:
close_data_sell.to_csv('./data/close_data_sell_5.csv', index=False)

In [ ]:
x = close_data_buy.drop(columns=['Y_BUY']).values
y = close_data_buy[['Y_BUY']].values

In [ ]:
features = x.shape[1]
features

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

In [ ]:
x_train = x_train.reshape(-1, features, 1)
x_test = x_test.reshape(-1, features, 1)

In [ ]:
dnn = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Dense(units=256, activation='relu', input_shape=(features,)),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')

])    

In [ ]:
dnn.compile(optimizer='adam', 
            loss='binary_crossentropy', 
            metrics=['accuracy'])

In [ ]:
dnn.summary()

In [ ]:
dnn.fit(x_train, y_train, epochs=50)